## Задача 1.
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [1]:
from scipy import stats

import numpy as np

import warnings

import matplotlib.pyplot as plt

import seaborn as sns

import pandas as pd

warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

### Тестирование гипотезы

In [2]:
data = pd.read_excel('AB_Test_Results.xlsx')

data.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [4]:
data.shape

(10000, 3)

In [5]:
data.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [6]:
data[data['VARIANT_NAME']=='control'].describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


In [7]:
data[data['VARIANT_NAME']=='variant'].describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


In [8]:
data[data['REVENUE'] > 0]

,USER_ID,VARIANT_NAME,REVENUE
13,2529,variant,2.15
49,6429,control,3.25
139,3331,variant,4.27
149,8110,variant,1.75
152,8607,control,2.99
...,...,...,...
9651,9928,variant,1.25
9711,6468,control,2.93
9724,5205,control,14.17
9725,702,variant,2.92


Проверяем данные на корректность содержимого

In [9]:
data.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

Проверяем данные на дублирование

In [10]:
data.VARIANT_NAME.value_counts()

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

In [11]:
data['VARIANT_NAME'].value_counts()

variant    5016
control    4984
Name: VARIANT_NAME, dtype: int64

Избавимся от некорретных строк. Посчитаем, сколько типов встречается у каждого пользователя.

In [25]:
v = data.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': pd.Series.nunique})

In [26]:
v.head()

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1


In [14]:
print(v)

      USER_ID  VARIANT_NAME
0           2             1
1           3             2
2           4             1
3           5             1
4           6             1
...       ...           ...
6319     9993             1
6320     9995             1
6321     9996             2
6322     9998             1
6323    10000             2

[6324 rows x 2 columns]


Из данной фильтрации видно, что в эксперименте присутствуют пользователи, которые попали как в контрольную, так и в тестируемую группы одновременно, в количестве 10000 - 6324 = 3676. Это состоявляет примерно 36,76% всех пользователей. Это является существенной частью, чтобы повлиять на итоги тестирования. Такие пользователи будут удалены из теста., равно как и дублирующиеся данные.

In [15]:
more_than_one_types = v.query('VARIANT_NAME > 1')

In [16]:
df_new = data[~data.USER_ID.isin(more_than_one_types.USER_ID)].sort_values('USER_ID')

In [18]:
df_new.shape

(6070, 3)

Истинное количество пользователей равно 6070.

In [20]:
A = df_new.query('VARIANT_NAME == "control"')

len(A)

3026

In [ ]:
B = df_new.query('VARIANT_NAME == "variant"')

len(B)

3044

Считаем, что выборки одинаковые!

### Проверка на нормальность распределения и применение статистических критериев

In [21]:
from scipy.stats import shapiro

alpha = 0.05

st = shapiro(data.REVENUE)

print('Distribution is {}normal\n'.format( {True:'not ', False:''}[st[1] < alpha]))

Distribution is not normal



Тест Шапиро показывает, что рассматриваемое распределение не нормальное, значит используем критерий Хи-квадрат Пирсона.

In [ ]:
data['REVENUE'].value_counts()

0.00     9848
1.25       10
1.01        6
3.25        6
0.04        5
         ... 
2.19        1
2.14        1
17.08       1
2.00        1
2.92        1
Name: REVENUE, Length: 101, dtype: int64

In [27]:
k1 = df_new[df_new['VARIANT_NAME']=='control']['REVENUE'].sum()

k2 = df_new[df_new['VARIANT_NAME']=='variant']['REVENUE'].sum()

print (k1, k2)

470.56000000000006 179.32


In [28]:
n1 = df_new[df_new['VARIANT_NAME']=='control'].shape[0]

n2 = df_new[df_new['VARIANT_NAME']=='variant'].shape[0]

print (n1, n2)

3026 3044


In [29]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), np.array([n1, n2]))

print('Results are ', 'z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =12.170, pvalue = 0.000


In [ ]:
if abs(z_pvalue) < 0.05:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We may reject the null hypothesis!


Непараметрический Хи-квадрат.

In [30]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =148.109, pvalue = 0.000


In [31]:
if abs(pvalue) < 0.05:
    print("We may reject the null hypothesis!")
else:
    print("We have failed to reject the null hypothesis")

We may reject the null hypothesis!


In [ ]:
Между группами имеются статистические различия, но контрольная группа успешнее.

Проверим непараметрический критерий Манна-Уитни.

In [32]:
df_new.groupby('VARIANT_NAME')['REVENUE'].describe()

,count,mean,std,min,25%,50%,75%,max
VARIANT_NAME,,,,,,,,
control,3026.0,0.155506,3.708620,0.0,0.0,0.0,0.0,196.01
variant,3044.0,0.058909,0.760344,0.0,0.0,0.0,0.0,23.04


In [ ]:
stats. mannwhitneyu(x=df_new[(df_new['VARIANT_NAME'] == 'control')&(df_new['REVENUE'] > 0)]['REVENUE'].values,
                                                  y=df_new[(df_new['VARIANT_NAME'] == 'variant')&(df_new['REVENUE'] > 0)]['REVENUE'].values)


MannwhitneyuResult(statistic=1292.0, pvalue=0.3431833825117172)

Получившееся p-value > 5% а это значит что между выборками нет статистических различий по Манну-Уитни

### Вывод:
Между двумя представленными выборками наблюдаются статистические различия. Следует дать следующую рекомендацию:
не рекомендуется применять тестовую версию, необходимо дополнительно проверить исходные данные на верное сплитование между пользователями, скорее следует провести А/А тест и проверить корректность теста.